## MLP Competition 124 - Lê Thanh Tuấn - 21115053120158

#### import lib

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import initializers
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#### config seed

In [15]:
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

#### load and process data

In [16]:
def load_and_process_data(train_path, target_path, test_path, test_size=0.2, random_state=42):
    X_train = pd.read_csv(train_path, low_memory=False).drop(columns=['ID'], errors='ignore')
    y_train = pd.read_csv(target_path)['TARGET']
    X_test = pd.read_csv(test_path, low_memory=False)
    
    test_ids = X_test.pop('ID') if 'ID' in X_test.columns else np.arange(len(X_test))

    for df in [X_train, X_test]:
        if 'tradeTime' in df.columns:
            df['tradeTime'] = pd.to_datetime(df['tradeTime'], errors='coerce')
        
        df.replace({
            'elevator': {1: 'has elevator', 0: 'no elevator'},
            'subway': {1: 'has subway', 0: 'no subway'},
            'buildingStructure': {1: 'unknown', 2: 'mixed', 3: 'brick and wood', 4: 'concrete', 5: 'steel', 6: 'steel-concrete composite'},
            'renovationCondition': {1: 'other', 2: 'rough', 3: 'Simplicity', 4: 'hardcover'},
            'buildingType': {1: 'tower', 2: 'bungalow', 3: 'combination of plate and tower', 4: 'plate'}
        }, inplace=True)
        
        if 'constructionTime' in df.columns:
            df['constructionTime'] = pd.to_numeric(df['constructionTime'], errors='coerce')

        if 'floor' in df.columns:
            df['floor'] = df['floor'].astype(str).str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

        # Convert Lat and Lng to numeric and calculate distance to the capital
        df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
        df['Lng'] = pd.to_numeric(df['Lng'], errors='coerce')

        capital_Lng = np.radians(116.4074)
        capital_Lat = np.radians(39.9042)
        df['distanceToCapital'] = np.arccos(
            np.sin(np.radians(df['Lat'])) * np.sin(capital_Lat) +
            np.cos(np.radians(df['Lat'])) * np.cos(capital_Lat) *
            np.cos(capital_Lng - np.radians(df['Lng']))
        ) * 6371.0088  # Earth's radius in kilometers

        if 'tradeTime' in df.columns and 'constructionTime' in df.columns:
            df['ageOfBuilding'] = df['tradeTime'].dt.year - df['constructionTime']

    # Handle missing values for 'elevator' and 'subway'
    for df in [X_train, X_test]:
        df['elevator'] = df['elevator'].fillna(0)
        df['subway'] = df['subway'].fillna(0)

    X_train = X_train.apply(pd.to_numeric, errors='coerce')
    X_test = X_test.apply(pd.to_numeric, errors='coerce')
    
    # Align columns between X_train and X_test to avoid mismatches
    common_cols = X_train.columns.intersection(X_test.columns)
    X_train, X_test = X_train[common_cols], X_test[common_cols]
    
    imputer = SimpleImputer(strategy="median")
    scaler = PowerTransformer()

    X_train = pd.DataFrame(scaler.fit_transform(imputer.fit_transform(X_train)), columns=common_cols)
    X_test = pd.DataFrame(scaler.transform(imputer.transform(X_test)), columns=common_cols)

    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
        X_train, y_train, test_size=test_size, random_state=random_state
    )

    return X_train_split, X_val_split, y_train_split, y_val_split, X_test, test_ids

#### build MLP model

In [17]:
def build_mlp_model(input_shape):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_shape,), kernel_initializer=initializers.HeNormal(seed=42)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_initializer=initializers.HeNormal(seed=42)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_initializer=initializers.HeNormal(seed=42)),
        Dense(1, kernel_initializer=initializers.HeNormal(seed=42))
    ])
    return model

#### compile and train MLP model

In [18]:
def compile_and_train_mlp(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=64):
    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    return model

#### train with RandomForest

In [19]:
def print_rmse(y_true, y_pred, model_name="Model"):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return rmse

def train_random_forest(X_train, y_train, X_val, y_val):
    rf_model = RandomForestRegressor(
        n_estimators=300,
        max_depth=20, 
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    )

    rf_model.fit(X_train, y_train)
    y_pred_val = rf_model.predict(X_val)
    val_rmse = print_rmse(y_val, y_pred_val, model_name="RandomForest")
    
    return rf_model, val_rmse

#### save output (submission)

In [20]:
def save_submission(y_pred, test_ids, filename='Latest_submission.csv'):
    submission = pd.DataFrame({
        "ID": test_ids,
        "TARGET": y_pred
    })
    submission.to_csv(filename, index=False)

#### usage

In [21]:
X_train_split, X_val_split, y_train_split, y_val_split, X_test_scaled, test_ids = load_and_process_data(
    'data/X_train.csv', 'data/y_train.csv', 'data/X_test.csv'
)

rf_model, rf_rmse = train_random_forest(X_train_split, y_train_split, X_val_split, y_val_split)

# Use Random Forest predictions as a new feature for MLP
rf_train_predictions = rf_model.predict(X_train_split).reshape(-1, 1)
rf_val_predictions = rf_model.predict(X_val_split).reshape(-1, 1)
rf_test_predictions = rf_model.predict(X_test_scaled).reshape(-1, 1)

# Append RF predictions to training and validation sets
X_train_split_with_rf = np.hstack((X_train_split, rf_train_predictions))
X_val_split_with_rf = np.hstack((X_val_split, rf_val_predictions))
X_test_with_rf = np.hstack((X_test_scaled, rf_test_predictions))

# Build and train the improved MLP model using RandomForest predictions as an additional feature
mlp_model = build_mlp_model(X_train_split_with_rf.shape[1])
mlp_model = compile_and_train_mlp(mlp_model, X_train_split_with_rf, y_train_split, X_val_split_with_rf, y_val_split)

mlp_model.save('weight/mlp_model.h5')
print("Model saved successfully to 'weight/mlp_model.h5'.")

# Predict using the trained MLP model on the validation set
y_pred_val_mlp = mlp_model.predict(X_val_split_with_rf).flatten()
mlp_rmse = print_rmse(y_val_split, y_pred_val_mlp, model_name="MLP with RF Feature")

# Predict on the test set using the trained MLP model
y_pred_test_mlp = mlp_model.predict(X_test_with_rf).flatten()

save_submission(y_pred_test_mlp, test_ids, filename='Latest_submission.csv')

d:\ProgramFiles\Anaconda\envs\ai_class\lib\site-packages\numpy\core\_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
d:\ProgramFiles\Anaconda\envs\ai_class\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


RandomForest RMSE: 53.35641146058034
Epoch 1/50
3189/3189 [==============================] - 8s 2ms/step - loss: 8772.5723 - val_loss: 3910.9028 - lr: 0.0010
Epoch 2/50
3189/3189 [==============================] - 7s 2ms/step - loss: 5775.0649 - val_loss: 3348.7214 - lr: 0.0010
Epoch 3/50
3189/3189 [==============================] - 7s 2ms/step - loss: 5192.0151 - val_loss: 7117.9937 - lr: 0.0010
Epoch 4/50
3189/3189 [==============================] - 7s 2ms/step - loss: 4991.5068 - val_loss: 10288.2109 - lr: 0.0010
Epoch 5/50
3189/3189 [==============================] - 7s 2ms/step - loss: 4998.6626 - val_loss: 9924.1709 - lr: 0.0010
Epoch 6/50
3189/3189 [==============================] - 7s 2ms/step - loss: 4565.2534 - val_loss: 5583.8296 - lr: 5.0000e-04
Epoch 7/50
3189/3189 [==============================] - 7s 2ms/step - loss: 4474.7705 - val_loss: 4443.5601 - lr: 5.0000e-04
Model saved successfully to 'weight/mlp_model.h5'.
MLP with RF Feature RMSE: 57.86814782965359
